In [1]:
import pyaudio
import wave
import whisper
import webrtcvad
import threading
from dotenv import load_dotenv
load_dotenv()

# Constants
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK_DURATION_MS = 30
VAD_PADDING_DURATION_MS = 2000
CHUNK_SIZE = int(RATE * CHUNK_DURATION_MS / 1000)
VAD_PADDING_CHUNKS = int(VAD_PADDING_DURATION_MS / CHUNK_DURATION_MS)
KEY_PHRASE = "Hey Grimoire"
MAX_RECORD_DURATION_MS = 10000
MAX_RECORD_CHUNKS = int(MAX_RECORD_DURATION_MS / CHUNK_DURATION_MS)

# Initialize VAD and PyAudio
vad = webrtcvad.Vad(1)
audio = pyaudio.PyAudio()

# Initialize global variables for conversation history
conversation_history = []

def ask_llm(question):
    """
    Simulated function to interact with an LLM and manage conversation history.
    """
    global conversation_history
    conversation_history.append(f"User: {question}")
    response = "This is a simulated LLM response."
    conversation_history.append(f"GPT: {response}")

    # Truncate history if it exceeds a certain size
    max_history = 5  # for example, keep the last 5 exchanges
    if len(conversation_history) > max_history * 2:
        conversation_history = conversation_history[-max_history * 2:]

    return response

def record_after_keyphrase():
    """
    Record audio after the key phrase is detected.
    """
    stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK_SIZE)
    frames = []
    num_silent_chunks = 0
    num_recorded_chunks = 0
    keyphrase_detected = False

    while num_recorded_chunks < MAX_RECORD_CHUNKS:
        try:
            chunk = stream.read(CHUNK_SIZE)
            is_speech = vad.is_speech(chunk, RATE)
            # ... [rest of your loop] ...
        except IOError as e:
            if e.errno == pyaudio.paInputOverflowed:
                print("Input overflowed, skipping this chunk.")
                chunk = '\x00' * CHUNK_SIZE  # Dummy chunk of silent audio

        if not keyphrase_detected:
            if is_speech:
                audio_data = b''.join(frames)
                temp_file = "temp.wav"
                wf = wave.open(temp_file, 'wb')
                wf.setnchannels(CHANNELS)
                wf.setsampwidth(audio.get_sample_size(FORMAT))
                wf.setframerate(RATE)
                wf.writeframes(audio_data)
                wf.close()

                model = whisper.load_model("base")
                result = model.transcribe(temp_file)
                if KEY_PHRASE.lower() in result["text"].lower():
                    keyphrase_detected = True
                    frames = []  # Start fresh frames
            else:
                frames = []  # Clear frames if no speech detected
        else:
            num_recorded_chunks += 1
            if not is_speech:
                num_silent_chunks += 1
                if num_silent_chunks >= VAD_PADDING_CHUNKS:
                    break
            else:
                num_silent_chunks = 0

    stream.stop_stream()
    stream.close()
    return b''.join(frames)

def transcribe_audio(audio_data):
    """
    Transcribe audio data using Whisper.
    """
    model = whisper.load_model("base")
    result = model.transcribe(audio_data)
    return result["text"]

def listen_and_transcribe():
    """
    Continuously listen for the key phrase and transcribe the speech.
    """
    while True:
        audio_data = record_after_keyphrase()
        question = transcribe_audio(audio_data)
        if question:
            print("Question:", question)
            response = ask_llm(question)
            print("Response:", response)

# Run the process in a separate thread
thread = threading.Thread(target=listen_and_transcribe)
thread.start()


/opt/homebrew/lib/python3.11/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Exception in thread Thread-4 (listen_and_transcribe):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.11/3.11.6_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/opt/homebrew/Cellar/python@3.11/3.11.6_1/Frameworks/Python.framework/Versions/3.11/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/bl/nkj7dq8s6m16hf4zmr47vkfr0000gn/T/ipykernel_75118/2567227741.py", line 101, in listen_and_transcribe
  File "/var/folders/bl/nkj7dq8s6m16hf4zmr47vkfr0000gn/T/ipykernel_75118/2567227741.py", line 53, in record_after_keyphrase
  File "/opt/homebrew/lib/python3.11/site-packages/pyaudio/__init__.py", line 570, in read
    return pa.read_